In [16]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
from tabulate import tabulate
pd.set_option("display.max_columns", None)


from variables import fecha
from cfg import postgresql_config

In [2]:
db = create_engine(postgresql_config)
conn = db.connect()

In [3]:
def traer_interanual(region):
    """
    Trae la tabla interanual de la base de datos
    """
    query = f"""
    SELECT * FROM indice_aperturas 
    WHERE date_part('month', fecha) = date_part('month', CURRENT_DATE) -1
    and region = '{region}'
    ORDER BY fecha DESC LIMIT 2;
    """
    df = pd.read_sql_query(query,con=db)
    
    return df
    



In [17]:
def print_interanual(region):
    df_interanual = traer_interanual(region)
    df_interanual.head()
    for column in df_interanual.columns[2::]:
        if column != 'region':
            df_interanual[column] = df_interanual[column].astype(float)

    for column in df_interanual.columns[2::]:
        if column != 'region':
            df_interanual[column] = ((df_interanual[column][0] - df_interanual[column][1]) / df_interanual[column][1]) * 100

    fecha = f'{df_interanual["fecha"][0].strftime("%B")} {df_interanual["fecha"][0].year}'
    df_interanual = df_interanual.dropna(axis=1)
    df_interanual = df_interanual.drop(columns=['fecha', 'index', 'region'])
    df_interanual_final = df_interanual
    df_interanual_final.iloc[0,:]
    df = pd.DataFrame(df_interanual_final.iloc[0,:])
    df = df.reset_index()
    df = df.rename(columns={'index': 'Indice Apertura', 0: 'variacion'})
    print(f'Indice interanual de aperturas de {fecha}')

    df2=df.nlargest(5,['variacion'])
    
    print(tabulate(df2, headers='keys', tablefmt='psql', showindex=False))

In [18]:

print_interanual('Region GBA')


Indice interanual de aperturas de October 2022
+----+---------------------------------+-------------+
|    | Indice Apertura                 |   variacion |
|----+---------------------------------+-------------|
| 17 | prendas_de_vestir_y_materiales  |     128.876 |
|  8 | verduras_tubérculos_y_legumbres |     122.759 |
| 16 | prendas_de_vestir_y_calzado     |     122.353 |
| 14 | bebidas_alcohólicas             |     113.095 |
|  6 | aceites_grasas_y_manteca        |     110.525 |
+----+---------------------------------+-------------+
